<a href="https://colab.research.google.com/github/GenaroHacker/crystal_clear/blob/main/filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/GenaroHacker/crystal_clear.git
%run crystal_clear/sql.ipynb

Cloning into 'crystal_clear'...
remote: Enumerating objects: 109, done.
remote: Counting objects: 100% (109/109), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 109 (delta 59), reused 11 (delta 1), pack-reused 0
Receiving objects: 100% (109/109), 34.71 KiB | 1.65 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [ ]:
#@title Finctions

def delete_paradigm(paradigm_id):
  list_of_tuples = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
  tablename_todelete = ""
  for paradigm in list_of_tuples:
    if paradigm[0] == paradigm_id:
      tablename_todelete = paradigm[1]
  RunCommand("BaseParadigms","DROP TABLE " + tablename_todelete)
  RemoveRecord("BaseParadigms","DELETE FROM TABLE_PARADIGMS WHERE ID=" + str(paradigm_id))

def sort_paradigm(paradigm_id):
  list_of_tuples = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
  tablename_tosort = ""
  for paradigm in list_of_tuples:
    if paradigm[0] == paradigm_id:
      tablename_tosort = paradigm[1]
  #Read all records
  list_of_tuples = ReadRecords("BaseParadigms",tablename_tosort)
  prdgms_to_sort = []
  for record in list_of_tuples:
    prdgms_to_sort.append(record[1])
  def manual_sort(items_to_sort):
    import functools
    def ask_user_cmp(item1, item2):
        while True:
            print(f"[{item1}](1) or [{item2}](2) ?")
            cmp = input("--> ")
            if cmp == "1":
                return 1
            if cmp == "2":
                return -1
            print("1 or 2, please!")
    ask_user_key = functools.cmp_to_key(ask_user_cmp)
    items_to_sort.sort(key=ask_user_key, reverse=True)
    return items_to_sort
  prdgms_sorted = manual_sort(prdgms_to_sort)
  RunCommand("BaseParadigms","DELETE FROM " + tablename_tosort)
  RunCommand("BaseParadigms","DELETE FROM sqlite_sequence WHERE name='" + tablename_tosort + "'")
  for phrase in prdgms_sorted:
    InsertRecord("BaseParadigms","INSERT INTO " + tablename_tosort + " VALUES (NULL,'" + phrase + "')")

def update_phrase(paradigm_id, phrase_id, new_phrase):
  list_of_tuples = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
  tablename = ""
  for paradigm in list_of_tuples:
    if paradigm[0] == paradigm_id:
      tablename = paradigm[1]
  UpdateRecord("BaseParadigms","UPDATE " + tablename + " SET PHRASE='" + new_phrase + "' WHERE ID=" + str(phrase_id))

def delete_phrase(paradigm_id,phrase_id):
  list_of_tuples = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
  tablename = ""
  for paradigm in list_of_tuples:
    if paradigm[0] == paradigm_id:
      tablename = paradigm[1]
  last_record_id = ReadLastRecord("BaseParadigms",tablename)[0]
  RemoveRecord("BaseParadigms","DELETE FROM " + tablename + " WHERE ID=" + str(phrase_id))
  for id in range(phrase_id + 1,last_record_id + 1):
    UpdateRecord("BaseParadigms","UPDATE " + tablename + " SET ID='" + str(id-1) + "' WHERE ID=" + str(id))


In [ ]:
#delete_paradigm(2)
#sort_paradigm(3)
#update_phrase(3,1,"lol")
#delete_phrase(1,1)

In [ ]:
#@title Execution
def FILTER_MODULE_MAIN_FUNC():
  update_table_paradigms()
  print("Select the paradigm you want to filter or type 0 to select a random paradigm")
  list_of_paradigms = ReadRecords("BaseParadigms","TABLE_PARADIGMS")
  print_table(list_of_paradigms)

  ids = []
  for i in list_of_paradigms:
    ids.append(i[0])


  user_choice = ""
  while type(user_choice) != int:
    try:
      user_choice = int(input())
      if user_choice not in ids and user_choice != 0:
        user_choice = ""
        int("I just hope that no ValueError occurs")
    except ValueError:
      print("Only IDs available in the table or 0 are allowed")
  if user_choice == 0:
    from random import choice
    user_choice = choice(ids)
    
  selected_paradigm_id = user_choice
  tablename = list_of_paradigms[selected_paradigm_id-1][1]
  print()
  print(tablename)
  list_of_phrases = ReadRecords("BaseParadigms",tablename)
  print_table(list_of_phrases)

  print()
  print("What do you want to do with [" + tablename + "]?")

  #delete_paradigm(2)
  #sort_paradigm(3)
  #update_phrase(3,1,"lol")
  #delete_phrase(1,1)

  print(" 1 --> sort paradigm")
  print(" 2 --> update phrase")
  print(" 3 --> delete phrase")
  print(" 4 --> delete paradigm")
  selected_action = input()
  while selected_action not in ("1", "2", "3", "4"):
    print("Option not available.")
    selected_action = input()

  if selected_action == "1":
    sort_paradigm(selected_paradigm_id)
    pass

  elif selected_action == "2":
    print()
    print("Select the phrase you want to update, or select 0 to cycle through all phrases.")
    print_table(list_of_phrases)

    options = ["0"]
    for phrase in list_of_phrases:
      options.append(str(phrase[0]))

    selected_phrase_to_update = input()
    while selected_phrase_to_update not in options:
      print("Phrase not available.")
      selected_phrase_to_update = input()

    if selected_phrase_to_update == "0":
      for phrase in list_of_phrases:
        print()
        print("from")
        print(phrase[1])
        print("to")
        new_phrase = input()
        if new_phrase == "":
          pass
        else:
          update_phrase(selected_paradigm_id,phrase[0],new_phrase)
    else:
      print()
      print("from")
      print(list_of_phrases[int(selected_phrase_to_update)-1][1])
      print("to")
      new_phrase = input()
      if new_phrase == "":
        pass
      else:
        update_phrase(selected_paradigm_id,list_of_phrases[int(selected_phrase_to_update)-1][0],new_phrase)

  elif selected_action == "3":
    print()
    print("Select phrase to delete,")
    print("or select 0 to cycle through all phrases,")
    print("or [END] to break.")
    print_table(list_of_phrases)

    options = ["0","end"]
    for phrase in list_of_phrases:
      options.append(str(phrase[0]))

    selected_phrase_to_delete = input().lower()
    while selected_phrase_to_delete not in options:
      print("Phrase not available.")
      selected_phrase_to_update = input().lower()

    if selected_phrase_to_delete == "end":
      pass

    elif selected_phrase_to_delete == "0":
      for phrase in list_of_phrases:
        print()
        print("Delete [" + phrase[1] + "]?")
        print("y/n")
        users_delete_choice = input()
        if users_delete_choice.lower() == "y":
          delete_phrase(selected_paradigm_id,phrase[0])
    else:
      delete_phrase(selected_paradigm_id,int(selected_phrase_to_delete))

  elif selected_action == "4":
    print()
    tablename_by_user = input("Write [" + tablename + "] to delete the paradigm.")
    if tablename == tablename_by_user:
      delete_paradigm(selected_paradigm_id)

Table already exists!!!!

Directivo
Coach
Gracioso
Select the paradigm you want to filter or type 0 to select a random paradigm
Table already exists!!!!

Directivo
Coach
Gracioso
1 | Directivo | 2 | 0 | 
2 | Coach     | 1 | 0 | 
3 | Gracioso  | 0 | 0 | 
